In [25]:
import snap
import json
import collections
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import numpy as np

In [2]:
business_path = '../yelp/yelp_academic_dataset_business.json'
user_path = '../yelp/yelp_academic_dataset_user.json'
review_path = '../yelp/yelp_academic_dataset_review.json'
tip_path = '../yelp/yelp_academic_dataset_tip.json'
review_small_path = '../yelp/review_small.json'
user_small_path = '../yelp/user_small.json'
business_small_path = '../yelp/business_small.json'
tip_small_path = '../yelp/tip_small.json'

In [5]:
# def find_reviews(path, path2):
#     print 'begin parsing reviews and tips...'
#     business_reviews = collections.defaultdict(list) # bid: [(uid, date)]
#     reviews = {} # review_id: (b_id, u_id, date)
#     with open(path) as json_file:
#         for line in json_file:
#             raw = json.loads(line)
#             reviews[raw['review_id']] = (raw['business_id'], raw['user_id'], raw['date'])
#             business_reviews[raw['business_id']].append((raw['user_id'], raw['date']))
            
#     # path2: tip
#     with open(path2) as json_file:
#         for line in json_file:
#             raw = json.loads(line)
#             business_reviews[raw['business_id']].append((raw['user_id'], raw['date']))
#     print 'done.'
#     return business_reviews, reviews

In [6]:
# def find_friends(path):
#     print 'begin parsing users...'
#     user_friends = collections.defaultdict(list) # {uid : [friends]}
#     with open(path) as json_file:
#         for line in json_file:
#             raw = json.loads(line)
#             if raw['friends'] != 'None':
#                 user_friends[raw['user_id']] = raw['friends'].split(', ')
#     print 'done.'
#     return user_friends

In [7]:
# def find_elites(path):
#     print 'begin parsing users...'
# #     user_friends = collections.defaultdict(list) # {uid : [friends]}
#     elite_class = {}
#     with open(path) as json_file:
#         for line in json_file:
#             raw = json.loads(line)
# #             if raw['friends'] != 'None':
# #                 user_friends[raw['user_id']] = raw['friends'].split(', ')
#             if raw['elite'] != 'None':
#                 elite_years = raw['elite'].split(', ')
#                 if len(elite_years) > 0:
#                     elite_class[raw['user_id']] = True
#                 else:
#                     elite_class[raw['user_id']] = False
#     print 'done.'
#     return elite_class  

In [9]:
def find_users(path):
    print 'begin parsing users...'
    users = collections.defaultdict() # uid: [review_count, year, friends, elite, 
    #average_stars, useful, funny, cool, fans]
    
    with open(path) as json_file:
        for line in json_file:
            raw = json.loads(line)
            year = int(raw['yelping_since'][:4])

            users[raw['user_id']] = (raw['review_count'], year, len(raw['friends']),len(raw['elite']),raw['average_stars'],
                                     raw['useful'],raw['funny'],raw['cool'],raw['fans'])

    print 'done.'
    return users

In [40]:
def nx_gather_business_cascades_feats(path, business_reviews, user_friends, elite_class,user_feats):
    test = 0
    print 'begin generating graphs...'
    with open(path) as json_file:
        for line in json_file:
            raw = json.loads(line)
            city = raw['city']
            bid = raw['business_id']
            G, nid2int, class_map, feats = nx_construct_local_graph_feats(business_reviews[bid], user_friends, elite_class,user_feats)
            
            if len(list(G.node)) < 15:
                continue
            
            test += 1
            print test
#             data1 = nx.readwrite.json_graph.node_link_data(G)
#             with open('GraphSAGE_input/' + bid + '-id_map.json', 'w') as fmap:
#                 json.dump(nid2int, fmap)
#             with open('GraphSAGE_input/' + bid + '-G.json', 'w') as fg:
#                 json.dump(data1, fg)
#             with open('GraphSAGE_input/' + bid + '-class_map.json', 'w') as fg:
#                 json.dump(class_map, fg)

            np.save('GraphSAGE_input/' + bid + '-feats', feats)
                
            if test > 3:
                break
    print 'done.'

In [15]:
def nx_construct_local_graph_feats(business_review, user_friends, elite_class,user_feats):
    G = nx.Graph()

    cleaned = {} # uid : time
    nid2int = {}
    class_map = {}
    G = nx.Graph()
    for uid, date in business_review:
        if len(user_friends[uid]) > 0:
            cleaned[uid] = date
#             uid2node[uid] =  G.AddNode(-1)
    for uid in cleaned:
        for friend in user_friends[uid]:
            if friend in cleaned and cleaned[friend] > cleaned[uid]:
                u_label = [1] if uid in elite_class else [0]
                f_label = [1] if friend in elite_class else[0]
                G.add_node(uid, label=u_label)
                G.add_node(friend, label=f_label)
                G.add_edge(uid, friend)
                class_map[uid] = u_label[0]
                class_map[friend] = f_label[0]
    mapped_int = 0
    feats = []
    for idx, nid in enumerate(list(G.node)):
        if idx >= len(list(G.node)) * 0.85:
            G.node[nid]['test'] = True
            G.node[nid]['val'] = False
        elif idx >= len(list(G.node)) * 0.7:
            G.node[nid]['val'] = True
            G.node[nid]['test'] = False
        else:
            G.node[nid]['test'] = False
            G.node[nid]['val'] = False            
        nid2int[nid] = mapped_int
        
        # feats
        feats.append(user_feats[nid])
        
        mapped_int += 1
    return G, nid2int, class_map, np.array(feats)

In [37]:
# import pickle
# user_friends = find_friends(user_small_path)
# business_reviews, reviews = find_reviews(review_small_path, tip_small_path)
# elite_class = find_elites(user_small_path)
# with open('business_reviews.pkl', 'wb') as f:
#     pickle.dump(business_reviews, f)""
# with open('user_friends.pkl', 'wb') as f:
#     pickle.dump(user_friends, f)
# with open('elite_class.pkl', 'wb') as f:
#     pickle.dump(elite_class, f)

# user_feats = find_users(user_path)
# with open('user-feats.pkl', 'wb') as f:
#     pickle.dump(user_feats, f)

begin parsing users...
done.
begin parsing reviews and tips...
done.
begin parsing users...
done.
begin parsing users...
done.


In [38]:
business_reviews = business_reviews = collections.defaultdict(list)
user_friends = collections.defaultdict(list) # {uid : [friends]}
with open('business_reviews.pkl', 'rb') as f:
    business_reviews = pickle.load(f)
with open('user_friends.pkl', 'rb') as f:
    user_friends = pickle.load(f)
with open('elite_class.pkl', 'rb') as f:
    elite_class = pickle.load(f)

with open('user-feats.pkl', 'wb') as f:
    user_feats = pickle.load(f)

In [41]:
nx_gather_business_cascades_feats(business_path, business_reviews, user_friends, elite_class,user_feats)

begin generating graphs...
1
2
3
4
done.
